# 🚀 ZeroLang Training Notebook

Train an AI model to generate WebAssembly (WAT) from natural language.

**Requirements:**
- GPU Runtime (A100 recommended)
- ~10GB disk space
- ~30 min training time

**Quick Start:**
1. Enable GPU: Runtime → Change runtime type → A100
2. Run all cells (Ctrl+F9)

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv

In [ ]:
# Install dependencies
!pip install -q torch transformers datasets peft accelerate bitsandbytes wandb

In [ ]:
# Clone repository
!git clone https://github.com/project-zero-git/zerolang.git
%cd zerolang

## 2. Prepare Data

In [ ]:
# Check data
!wc -l data/*.jsonl

In [ ]:
# Preview training data
import json
with open('data/train_chatml_v2.jsonl') as f:
    sample = json.loads(f.readline())
    print("User:", sample['messages'][1]['content'][:100])
    print("Assistant:", sample['messages'][2]['content'][:200])

## 3. Train Model

Choose model based on GPU:
- **T4 (16GB)**: `qwen-3b` or `qwen-1.5b`
- **A100 (40GB)**: `qwen-7b` or `llama-8b`
- **A100 (80GB)**: `qwen-14b`

In [ ]:
# Training configuration
MODEL = "qwen-7b"  # Change based on GPU
EPOCHS = 10
BATCH_SIZE = 4
MAX_LENGTH = 1024

In [ ]:
# Start training
!python training/train_cloud.py \
    --model {MODEL} \
    --epochs {EPOCHS} \
    --batch-size {BATCH_SIZE} \
    --max-length {MAX_LENGTH} \
    --output models/zerolang-{MODEL}

## 4. Test Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Load model
model_path = f"models/zerolang-{MODEL}"
base_model = {
    "qwen-7b": "Qwen/Qwen2.5-7B-Instruct",
    "qwen-3b": "Qwen/Qwen2.5-3B-Instruct",
    "llama-8b": "meta-llama/Llama-3.1-8B-Instruct",
}[MODEL]

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, model_path)
model.eval()
print("Model loaded!")

In [ ]:
def generate_wat(instruction, max_tokens=512):
    messages = [
        {"role": "system", "content": "You are ZeroLang, an AI that generates optimized WebAssembly (WAT) code. Given a natural language description of a function, output valid WAT code that implements it. Output only the WAT code, no explanations."},
        {"role": "user", "content": instruction},
    ]
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("assistant")[-1].strip()

In [ ]:
# Test generation
test_prompts = [
    "Implement: int add(int a, int b)",
    "Implement: int factorial(int n)",
    "Implement: void swap(int *a, int *b)",
]

for prompt in test_prompts:
    print(f"\n{'='*60}")
    print(f"Input: {prompt}")
    print(f"{'='*60}")
    wat = generate_wat(prompt)
    print(wat[:500] if len(wat) > 500 else wat)

## 5. Save & Download Model

In [ ]:
# Zip model for download
!zip -r zerolang-model.zip models/zerolang-{MODEL}

In [ ]:
# Download (Colab)
from google.colab import files
files.download('zerolang-model.zip')

## Optional: Push to HuggingFace Hub

In [ ]:
# Uncomment and run to push to HF Hub
# from huggingface_hub import login
# login()  # Enter your token
# 
# model.push_to_hub("your-username/zerolang-v1")
# tokenizer.push_to_hub("your-username/zerolang-v1")